In [ ]:
import os

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import keras_tuner as kt

import gc

# Importing and transforming dataset

In [ ]:
def label_counter(inpt, data_name):
    c = 0
    u = 0
    for i in inpt:
        if i == 1:
            c = c+1
        elif i == 0:
            u = u + 1
    print(f'{data_name} --> Congested: {c}, Uncongested: {u}')

def import_dataset(split_seed):
    
    #First dataset
    congested = pl.read_csv('/kaggle/input/traffic-images-data/model2_congested_traffic_dataset.csv')
    
    #Second dtaset
    uncongested = pl.read_csv('/kaggle/input/traffic-images-data/model2_uncongested_traffic_dataset.csv')
    
    #Combining first and second dataset
    df = pl.concat([congested, uncongested], how="vertical")
    
    y = df[:, 0] # Getting labels as series
    y = y.to_numpy()
    
    df = df[:, 1:]/255.0 # Normalizing pixels value to range of 0 to 1
    df = df.to_numpy().reshape(-1, 222, 296, 1) #reshaping to 296 by 222

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.1, random_state=split_seed) #42

    # Split train data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=split_seed)
    
    label_counter(y_train, 'y_train')
    label_counter(y_test, 'y_test')
    label_counter(y_val, 'y_val')
    
    return X_train, y_train, X_test, y_test, X_val, y_val

X_train, y_train, X_test, y_test, X_val, y_val = import_dataset(25)

# Exploring dataset

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define class labels
class_labels = {0: 'Congested images', 1: 'Uncongested images'}

# Map class labels for visualization
train_classes = np.array([class_labels[label] for label in y_train])
test_classes = np.array([class_labels[label] for label in y_test])
val_classes = np.array([class_labels[label] for label in y_val])

# Calculate class frequencies
train_class_counts = np.unique(train_classes, return_counts=True)
test_class_counts = np.unique(test_classes, return_counts=True)
val_class_counts = np.unique(val_classes, return_counts=True)

# Create bar chart data
data = [
    go.Bar(x=train_class_counts[0], y=train_class_counts[1], name='Train'),
    go.Bar(x=test_class_counts[0], y=test_class_counts[1], name='Test'),
    go.Bar(x=val_class_counts[0], y=val_class_counts[1], name='Validation')
]

# Set layout
layout = go.Layout(
    title='Occurrences of Binary Classes',
    xaxis=dict(title='Class'),
    yaxis=dict(title='Count'),
    barmode='group'
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show the bar chart
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define class labels
class_labels = {
    0: 'Congested',
    1: 'Uncongested'
}

# Calculate total class counts
class_counts = {
    class_labels[0]: np.sum([np.sum(y_train == 0), np.sum(y_test == 0), np.sum(y_val == 0)]),
    class_labels[1]: np.sum([np.sum(y_train == 1), np.sum(y_test == 1), np.sum(y_val == 1)])
}

# Create bar chart data
data = [
    go.Bar(x=list(class_counts.keys()), y=list(class_counts.values()))
]

# Set layout
layout = go.Layout(
    title='Total Occurrences of Binary Classes',
    xaxis=dict(title='Class'),
    yaxis=dict(title='Count')
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show the bar chart
fig.show()

In [ ]:
def image_visualizer(X, y, row_number):
    X = X[row_number]
    y = y[row_number]

    plt.imshow(X, cmap='gray')
    lbl = None
    if y == 0:
        lbl = 'Uncongested'
    elif y == 1:
        lbl = 'Congested'
    plt.title(lbl)
    plt.show()

image_visualizer(X_train, y_train, 5)
image_visualizer(X_train, y_train, 225)
image_visualizer(X_train, y_train, 512)
image_visualizer(X_train, y_train, 995)

# Model hyperparameter tuning

In [ ]:
import tensorflow as tf
import keras_tuner as kt

def model_builder(hp):
    model = keras.Sequential()
    
    model.add(keras.layers.Conv2D(hp.Int('filter_1', min_value=5, max_value=100, step=10), hp.Choice('filter_size_1', values=[3, 4, 5]), activation='relu', input_shape=(222, 296, 1)))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_1', values=[1, 2])))
    
    model.add(keras.layers.Conv2D(hp.Int('filter_2', min_value=5, max_value=70, step=5), hp.Choice('filter_size_2', values=[1, 2]), activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_2', values=[1, 2])))
    
    model.add(keras.layers.Conv2D(hp.Int('filter_3', min_value=5, max_value=70, step=5), hp.Choice('filter_size_3', values=[1, 2]), activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_3', values=[1, 2])))
    
    model.add(keras.layers.Conv2D(hp.Int('filter_4', min_value=5, max_value=70, step=5), hp.Choice('filter_size_4', values=[1, 2]), activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_4', values=[1, 2])))
              
    model.add(keras.layers.Conv2D(hp.Int('filter_5', min_value=5, max_value=70, step=5), hp.Choice('filter_size_5', values=[1, 2]), activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_5', values=[1, 2])))
        
    model.add(keras.layers.Conv2D(hp.Int('filter_6', min_value=5, max_value=80, step=5), hp.Choice('filter_size_6', values=[1, 2]), activation='relu'))
    model.add(keras.layers.MaxPooling2D(hp.Choice('kernel_size_6', values=[1, 2])))
              
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=hp.Int('units_1', min_value=5, max_value=100, step=10), activation='relu'))
    
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout_rate))
    
    model.add(keras.layers.Dense(hp.Int('units_2', min_value=5, max_value=55, step=5), activation='relu'))
    
    model.add(keras.layers.Dense(2, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0003, 0.0001])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='/kaggle/working/',
                     project_name='congested_vs_uncongested_traffic_detector')
              
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train, epochs=15, validation_data=(X_val, y_val), callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Training optimum model on dataset

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
# Data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

history = model.fit(datagen.flow(X_train, y_train),
                    validation_data=(X_val, y_val),
                    epochs=60)

In [ ]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Re-instantiate the hypermodel with the optimal number of epochs

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(datagen.flow(X_train, y_train),
                    epochs=best_epoch,
                    validation_data=(X_val, y_val))

# Evaluating

In [ ]:
loss, accuracy = hypermodel.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

In [ ]:
index = 1
print(y_test[index])
hypermodel.predict(X_test[index].reshape(-1, 222, 296, 1))

In [ ]:
index = 108
print(y_test[index])
hypermodel.predict(X_test[index].reshape(-1, 222, 296, 1))

In [ ]:
index = 233
print(y_test[index])
hypermodel.predict(X_test[index].reshape(-1, 222, 296, 1))

In [ ]:
index = 154
print(y_test[index])
hypermodel.predict(X_test[index].reshape(-1, 222, 296, 1))

# Saving

In [ ]:
hypermodel.save('/kaggle/working/model')